# [TODO] implement implicit-euler using finite array of state functions and their derivatives

# [TODO] Compare the analytical solution of trajectory for gaussian case

In [2]:
import numpy as np

In [3]:
delta_t = 0.05
t0 = 0.0
k0 = 0.1

def get_psi_dpsi_ddpsi(x,t):
    _psi = np.exp(1.0j * (k0 * x - 0.5*k0*k0*t))
    _dpsi = 1.0j * k0 * _psi
    _ddpsi = - k0 * k0 * _psi
    return (_psi, _dpsi, _ddpsi)

from tdse.analytic import Gaussian1D, gradient_Gaussian1D, laplacian_Gaussian1D
def get_psi_dpsi_ddpsi(x,t):
    _psi = Gaussian1D(x,t,k0)
    _dpsi = gradient_Gaussian1D(x,t,k0)
    _ddpsi = laplacian_Gaussian1D(x,t,k0)
    return (_psi, _dpsi, _ddpsi)

from tdse.finite_difference import eval_deriv_on_equidistanced_grid
x_arr = np.linspace(-2,2,201)
def get_psi_dpsi_ddpsi(x, t):
    if not isinstance(x, np.ndarray): x = np.array(x)
    _psi = Gaussian1D(x_arr,t,k0)
    return eval_deriv_on_equidistanced_grid(x, x_arr, _psi, orders=[0,1,2], N_s=4)[:,0]


def get_psi(x, t):
#     return Gaussian1D(x_arr,t,k0)
    return np.exp(1.0j * (k0 * x - 0.5*k0*k0*t))

In [4]:
def prop_xx(xx_t, delta_t, x_arr, sf_arr_next, log=False, g_zero_thres=1e-11, xx_diff_thres=1e-13):
    
#     x_arr, sf_arr_next = farg
    
#     _t_next = t + delta_t
    
    _xx_t_k = xx_t

    while True:

#         _psi, _dpsi, _ddpsi = get_psi_dpsi_ddpsi(_xx_t_k, _t_next)
        _psi, _dpsi, _ddpsi = eval_deriv_on_equidistanced_grid(np.array([xx_t]), x_arr, sf_arr_next, orders=[0,1,2], N_s=4)[:,0]
        
        _g_k = xx_t - _xx_t_k + delta_t * (_dpsi / _psi).imag
        if log: print("xx_t_k: {:.16f} / g_k: {:.16f}".format(_xx_t_k, _g_k))

        if np.abs(_g_k) < g_zero_thres: break

        _psi_sq = (_psi.conj() * _psi).real
        _xx_t_k_next = _xx_t_k - ( (xx_t - _xx_t_k)*_psi_sq + delta_t * (_psi.conj()*_dpsi).imag ) / ( delta_t * (_psi.conj()*_ddpsi - _psi.conj() / _psi * (_dpsi*_dpsi)).imag - 1.0)
        assert np.abs(_xx_t_k_next - _xx_t_k) > xx_diff_thres
        _xx_t_k = _xx_t_k_next
    
    _xx_t_next = _xx_t_k
    
    return _xx_t_next

In [5]:
x_p_arr = np.array([0.0, 0.31, 0.6])

for x_p in x_p_arr:
    xx_t = x_p
    t = t0

    for _t_idx in range(5):
        
        _sf_arr_next = get_psi(x_arr, t+delta_t)
        xx_t = prop_xx(xx_t, delta_t, x_arr, _sf_arr_next, log=False)
        t = t + delta_t

        print("t: {:.15f} / xx_t: {:.15f}".format(t,xx_t))
    print("")

t: 0.050000000000000 / xx_t: 0.004999999999994
t: 0.100000000000000 / xx_t: 0.009999999999994
t: 0.150000000000000 / xx_t: 0.014999999999992
t: 0.200000000000000 / xx_t: 0.019999999999987
t: 0.250000000000000 / xx_t: 0.024999999999981

t: 0.050000000000000 / xx_t: 0.314999999999998
t: 0.100000000000000 / xx_t: 0.319999999999993
t: 0.150000000000000 / xx_t: 0.324999999999987
t: 0.200000000000000 / xx_t: 0.329999999999987
t: 0.250000000000000 / xx_t: 0.334999999999985

t: 0.050000000000000 / xx_t: 0.604999999999994
t: 0.100000000000000 / xx_t: 0.609999999999994
t: 0.150000000000000 / xx_t: 0.614999999999992
t: 0.200000000000000 / xx_t: 0.619999999999987
t: 0.250000000000000 / xx_t: 0.624999999999981

